# Creating a Grid Integrated Plant Design Problem using IDAES Surrogates

## Import necessary packages
To begin, we need to import necessary packages. The primary packages required are:
* Pyomo : Modeling the optimization problem
* IDAES: Surrogate modeling capabilities
* Tensorflow: load trained Keras surrogates that represent market simulations

In [17]:
#the rankine cycle is in a directory above this one, so modify the path to import it
#eventually you should be able to do something like: idaes.flowsheets.simple_rankine_cycle import*
import sys, os, json
sys.path.append(os.path.join(os.path.curdir,"../../../models/simple_case"))
from simple_rankine_cycle import *

#import pyomo and its objects
from pyomo.environ import ConcreteModel, SolverFactory, units, Var, \
    TransformationFactory, value, Block, Expression, Constraint, Param, \
    Objective, NonNegativeReals
import pyomo.environ as pyo

# Import IDAES components
from idaes.core import FlowsheetBlock
from idaes.core.util import get_solver

#Import IDAES SurrogateBlock
from idaes.surrogate.keras_surrogate import KerasSurrogate
from idaes.surrogate.sampling.scaling import OffsetScaler
from idaes.surrogate.surrogate_block import SurrogateBlock

#Import tensorflow for model loading
import tensorflow
import pandas as pd

## Load Keras Surrogates
The first step towards assembling our optimization model is to load the pre-trained Keras neural networks (available in this DISPATCHES repository) as well as associated meta-data saved in corresponding .json files. This meta data contains input and output labels, scaling information, and the input bounds used during training.

The keras models themselves represent the results of market simulations using the Prescient Production Cost Modeling open-source package. We previously trained neural network models to predict the following outputs based on market inputs:
* Annual Revenue \[MM\$\]: (the total amount of revenue our generator earned per year)
* Annual Number of Generator Startups
* Annual Zone Output \[hours\]: (the amount of time our generator spent at different power output intervals). For this neural network, we trained 11 different zone outputs (off,0-10% of pmax,10-20% of pmax,...,90-100% of pmax)

For market inputs, our trianing data consisted of 8 primary attributes:
* p_max (the nameplate plant capacity)
* p_min_multipler (the minimum operating output as a fration of p_max)
* ramp_rate_multiplier
* minimum_up_time
* minimum_dn_multiplier
* marginal_cost
* fixed_run_cost
* startup_profile

In [18]:
# load training meta-data for each surrogate. this includes things like scaling \
# information, labels, and input bounds
surrogate_dir = os.path.join(os.path.curdir,\
"../../train_market_surrogates/steady_state/surrogate_models/keras/models")
with open(os.path.join(surrogate_dir,"training_parameters_revenue.json"), 'rb') as f:
    rev_data = json.load(f)

with open(os.path.join(surrogate_dir,"training_parameters_zones.json"), 'rb') as f:
    zone_data = json.load(f)

with open(os.path.join(surrogate_dir,"training_parameters_nstartups.json"), 'rb') as f:
    nstartups_data = json.load(f)

# unpack scaling information for revenue
xm_rev = rev_data['xm_inputs']
xstd_rev = rev_data['xstd_inputs']
zm_rev = rev_data['zm_revenue']
zstd_rev = rev_data['zstd_revenue']

# unpack scaling information for number of startups
xm_nstart = nstartups_data['xm_inputs']
xstd_nstart = nstartups_data['xstd_inputs']
zm_nstartups = nstartups_data['zm_nstartups']
zstd_nstartups = nstartups_data['zstd_nstartups']

# unpack scaling information for duration at different operating outputs
xm_zones = zone_data['xm_inputs']
xstd_zones = zone_data['xstd_inputs']
zm_zones = zone_data['zm_zones']
zstd_zones = zone_data['zstd_zones']

# load surrogates from keras directories
keras_revenue = tensorflow.keras.models.load_model(os.path.join(surrogate_dir,'keras_revenue'), compile=False)
keras_nstartups = tensorflow.keras.models.load_model(os.path.join(surrogate_dir,'keras_nstartups'), compile=False)
keras_zones = tensorflow.keras.models.load_model(os.path.join(surrogate_dir,'keras_zones'), compile=False)

## Create IDAES Keras Surrogate Objects

Now that we have our keras models loaded we can create `KerasSurrogate` objects using IDAES. These objects wrap our trained Keras Sequential models and store the additional training information we loaded such as scaling. The `KerasSurrogate` object provides a consistent interface for handling surrogate models in IDAES.

Once we have a `KerasSurrogate`, we can use it to build a mathematical model on a `SurrogateBlock`. This is shown later in this notebook.

In [19]:
# revenue Keras surrogate
# load extra meta-data for the revenue surrogate
input_labels = rev_data['input_labels']
output_labels = rev_data['output_labels']
xmin = rev_data['xmin']
xmax = rev_data['xmax']

# create scaling objects for the neural network input and outputs
inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels,xm_rev))),
    factor_series = pd.Series(dict(zip(input_labels,xstd_rev))))

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels,[zm_rev]))),
    factor_series = pd.Series(dict(zip(output_labels,[zstd_rev]))))

# store input bounds in a dictionary where keys correspond to input labels and values are 2-length tuples
# of the lower and upper bound respectively
input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

# create the KerasSurrogate object using the keras neural network and associated training data
keras_revenue_surrogate = KerasSurrogate(keras_model=keras_revenue,
                                 input_labels=input_labels,
                                 output_labels=output_labels,
                                 input_bounds=input_bounds,
                                 input_scaler=inputs_scaler,
                                 output_scaler=outputs_scaler)

# nstartups Keras surrogate
input_labels = nstartups_data['input_labels']
output_labels = nstartups_data['output_labels']
xmin = nstartups_data['xmin']
xmax = nstartups_data['xmax']

inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels,xm_nstart))),
    factor_series = pd.Series(dict(zip(input_labels,xstd_nstart))))

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels,[zm_nstartups]))),
    factor_series = pd.Series(dict(zip(output_labels,[zstd_nstartups]))))

input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

keras_nstartups_surrogate = KerasSurrogate(keras_model=keras_nstartups,
                                 input_labels=input_labels,
                                 output_labels=output_labels,
                                 input_bounds=input_bounds,
                                 input_scaler=inputs_scaler,
                                 output_scaler=outputs_scaler)

# zone hour power output Keras surrogate
input_labels = zone_data['input_labels']
output_labels = zone_data['output_labels']
xmin = zone_data['xmin']
xmax = zone_data['xmax']

inputs_scaler = OffsetScaler(
    expected_columns=input_labels,
    offset_series = pd.Series(dict(zip(input_labels,xm_zones))),
    factor_series = pd.Series(dict(zip(input_labels,xstd_zones)))
    )

outputs_scaler = OffsetScaler(
    expected_columns=output_labels,
    offset_series = pd.Series(dict(zip(output_labels,zm_zones))),
    factor_series = pd.Series(dict(zip(output_labels,zstd_zones)))
    )

input_bounds={input_labels[i]: (xmin[i], xmax[i]) for i in range(len(input_labels))}

keras_zones_surrogate = KerasSurrogate(keras_model=keras_zones,
                                 input_labels=input_labels,
                                 output_labels=output_labels,
                                 input_bounds=input_bounds,
                                 input_scaler=inputs_scaler,
                                 output_scaler=outputs_scaler)

## Building the Rankine Cycle Model

We now build the rankine cycle model available from DISPATCHES. This cell creates a pyomo model and uses rankine cycle functions to construct a design flowsheet. More specifically, we construct a high level flowsheet to represent the plant design. We will later construct additional flowsheets that represent plant operation for different power outputs.

In [20]:
#rankine cycle parameters
heat_recovery=True
calc_boiler_eff=True
capital_payment_years=5
plant_lifetime=20
coal_price=30 #$/ton

m = ConcreteModel()

# Create capex plant
m.cap_fs = create_model(
    heat_recovery=heat_recovery,
    capital_fs=True, calc_boiler_eff=False)
m.cap_fs = set_inputs(m.cap_fs)
m.cap_fs = initialize_model(m.cap_fs)
m.cap_fs = close_flowsheet_loop(m.cap_fs)
m.cap_fs = add_capital_cost(m.cap_fs)

# capital cost (M$/yr)
cap_expr = m.cap_fs.fs.capital_cost/capital_payment_years

2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.boiler.control_volume: Initialization Complete
2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.boiler: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.turbine: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.pre_condenser.control_volume: Initialization Complete
2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.pre_condenser: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.condenser.control_volume: Initialization Complete
2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.condenser: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:34 [INFO] idaes.init.cap_fs.fs.bfw_pump.control_volume: Initialization Complete
2022-03-09 16:23:35 [INFO] idaes.init.cap_fs.fs.bfw_pump: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:35 [INFO

## Build the grid market surrogates

We now create variables to represent the previously defined market inputs such as name-plate capacity, ramp rate, marginal cost, etc...

We use the IDAES `SurrogateBlock` to build the neural network constraints by using the previously creates `KerasSurrogate` objects. Notably, the build function automatically performs the input and output scaling because we provided the scaling information to each `KerasSurrogate`.

In [21]:
########################################
#surrogate market inputs
########################################
m.pmax = Var(within=NonNegativeReals, bounds=(175,450)) #MW
m.pmax_con = Constraint(expr = m.pmax == 1.0*m.cap_fs.fs.net_cycle_power_output*1e-6)
m.pmin_multi = Var(within=NonNegativeReals, bounds=(0.15,0.45), initialize=0.3)
m.ramp_multi = Var(within=NonNegativeReals, bounds=(0.5,1.0), initialize=0.5)
m.min_up_time = Var(within=NonNegativeReals, bounds=(1.0,16.0), initialize=4.0)
m.min_dn_multi = Var(within=NonNegativeReals, bounds=(0.5,2.0), initialize=2.0)
m.marg_cst =  Var(within=NonNegativeReals, bounds=(5,30), initialize=5)
m.no_load_cst =  Var(within=NonNegativeReals, bounds=(0,2.5), initialize=1)
m.startup_cst = Var(within=NonNegativeReals, bounds=(0,136), initialize=1)

#actual generator values for minimum operating output, minimum down time, and true ramp rate
m.pmin = Expression(expr = m.pmin_multi*m.pmax) #MW
m.min_dn_time = Expression(expr = m.min_dn_multi*m.min_up_time) #hours
m.ramp_rate= Expression(expr =  m.ramp_multi*(m.pmax - m.pmin)) #MW/hr

In [22]:
######################################
#revenue surrogate
######################################
m.rev_surrogate = Var()
m.keras_revenue_surrogate = SurrogateBlock()
m.keras_revenue_surrogate.build_model(keras_revenue_surrogate,
                                      input_vars=[m.pmax,m.pmin_multi,m.ramp_multi,m.min_up_time,\
                                                 m.min_dn_multi,m.marg_cst,m.no_load_cst,m.startup_cst],
                                      output_vars = [m.rev_surrogate],
                                      formulation=KerasSurrogate.Formulation.REDUCED_SPACE)
#this is a smooth-max; it sets negative revenue to zero
m.revenue = Expression(expr=0.5*pyo.sqrt(m.rev_surrogate**2 + 0.001**2) + 0.5*m.rev_surrogate)

Setting bound of pmax to (177.5, 443.75).
Setting bound of pmin_multi to (0.15, 0.45).
Setting bound of ramp_multi to (0.5, 1.0).
Setting bound of min_up_time to (1.0, 16.0).
Setting bound of min_dn_multi to (0.5, 2.0).
Setting bound of marg_cst to (5.0, 30.0).
Setting bound of no_load_cst to (0.0, 2.5).
Setting bound of startup_cst to (0.0, 135.2230393).


In [23]:
#######################################
#nstartups surrogate
#######################################
m.nstartups_surrogate = Var()
m.keras_nstartups_surrogate = SurrogateBlock()
m.keras_nstartups_surrogate.build_model(keras_nstartups_surrogate,
                                       input_vars=[m.pmax,m.pmin_multi,m.ramp_multi,m.min_up_time,\
                                                 m.min_dn_multi,m.marg_cst,m.no_load_cst,m.startup_cst],
                                      output_vars = [m.rev_surrogate],
                                      formulation=KerasSurrogate.Formulation.REDUCED_SPACE)

m.nstartups = Expression(expr=0.5*pyo.sqrt(m.nstartups_surrogate**2 + 0.001**2) + 0.5*m.nstartups_surrogate)

Setting bound of pmax to (177.5, 443.75).
Setting bound of pmin_multi to (0.15, 0.45).
Setting bound of ramp_multi to (0.5, 1.0).
Setting bound of min_up_time to (1.0, 16.0).
Setting bound of min_dn_multi to (0.5, 2.0).
Setting bound of marg_cst to (5.0, 30.0).
Setting bound of no_load_cst to (0.0, 2.5).
Setting bound of startup_cst to (0.0, 135.2230393).


In [24]:
############################################
#zone surrogates
############################################
m.zone_hours_surrogate = Var(range(0,11))
m.keras_zones_surrogate = SurrogateBlock()
m.keras_zones_surrogate.build_model(keras_zones_surrogate,
                                   input_vars=[m.pmax,m.pmin_multi,m.ramp_multi,m.min_up_time,\
                                   m.min_dn_multi,m.marg_cst,m.no_load_cst,m.startup_cst],
                                   output_vars = m.zone_hours_surrogate,
                                   formulation=KerasSurrogate.Formulation.REDUCED_SPACE)


Setting bound of pmax to (177.5, 443.75).
Setting bound of pmin_multi to (0.15, 0.45).
Setting bound of ramp_multi to (0.5, 1.0).
Setting bound of min_up_time to (1.0, 16.0).
Setting bound of min_dn_multi to (0.5, 2.0).
Setting bound of marg_cst to (5.0, 30.0).
Setting bound of no_load_cst to (0.0, 2.5).
Setting bound of startup_cst to (0.0, 135.2230393).


## Build the Operation Flowsheets

We now create a flowsheet for each possible operating zone. Each flowsheet uses the market decisions defined above and we use the zone surrogates to predict the number of hours spent at each flowsheet's powr output.

In [25]:
off_fs = Block()
off_fs.fs = Block()
off_fs.fs.operating_cost = m.no_load_cst*m.pmax
off_fs.zone_hours = Expression(expr=0.5*pyo.sqrt(m.zone_hours_surrogate[0]**2 + 0.001**2) + 0.5*m.zone_hours_surrogate[0])
setattr(m, 'zone_{}'.format('off'), off_fs)

#Denote the scaled power output for each of the 10 zones (0 corresponds to pmin, 1.0 corresponds to pmax)
zone_outputs = [0.0,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,1.0]

#Create a surrogate flowsheet for each operating zone
op_zones = []
init_flag = 0
for (i,zone_output) in enumerate(zone_outputs):
    print("Creating instance ", i)
    op_fs = create_model(
        heat_recovery=heat_recovery,
        capital_fs=False,
        calc_boiler_eff=calc_boiler_eff)
    # Set model inputs for the capex and opex plant
    op_fs = set_inputs(op_fs)

    # Fix the p_max of op_fs to p of cap_fs for initialization
    op_fs.fs.net_power_max.fix(value(m.cap_fs.fs.net_cycle_power_output))

    #initialize with json. this speeds up model instantiation. it writes a json file \
    #for the first flowsheet which is used to initialize the next flowsheets
    if init_flag == 0:
        # Initialize the opex plant
        op_fs = initialize_model(op_fs)

        # save model state after initializing the first instance
        to_json(op_fs.fs, fname="initialized_state.json.gz",
                gz=True, human_read=True)
        init_flag = 1
    else:
        # Initialize the capex and opex plant
        from_json(op_fs.fs, fname="initialized_state.json.gz", gz=True)

    # Closing the loop in the flowsheet
    op_fs = close_flowsheet_loop(op_fs)
    op_fs = add_operating_cost(op_fs, coal_price=coal_price)

    # Unfix op_fs p_max and set constraint linking that to cap_fs p_max
    op_fs.fs.net_power_max.unfix()
    op_fs.fs.eq_p_max = Constraint(
        expr=op_fs.fs.net_power_max ==
        m.cap_fs.fs.net_cycle_power_output*1e-6
    )

    #Fix zone power output
    op_fs.fs.eq_fix_power = Constraint(expr=op_fs.fs.net_cycle_power_output*1e-6 == zone_output*(m.pmax-m.pmin) + m.pmin)

    #smooth max on zone hours (avoids negative hours)
    op_fs.zone_hours = Expression(expr=0.5*pyo.sqrt(m.zone_hours_surrogate[i+1]**2 + 0.001**2) + 0.5*m.zone_hours_surrogate[i+1])

    #unfix the boiler flow rate
    op_fs.fs.boiler.inlet.flow_mol[0].setlb(0.01)
    op_fs.fs.boiler.inlet.flow_mol[0].unfix()
    setattr(m, 'zone_{}'.format(i), op_fs)
    op_zones.append(op_fs)

#scale zone hours such that they add up to 8736 (if the surrogate is good, the unscaled will be pretty close to this)
m.zone_total_hours = sum(op_zones[i].zone_hours for i in range(len(op_zones))) + off_fs.zone_hours
for op_fs in op_zones:
    op_fs.scaled_zone_hours = Var(within=NonNegativeReals, bounds=(0,8736), initialize=100)
    # NOTE: scaled_hours_i = surrogate_i * 8736 / surrogate_total
    op_fs.con_scale_zone_hours = Constraint(expr = op_fs.scaled_zone_hours*m.zone_total_hours == op_fs.zone_hours*8736)
off_fs.scaled_zone_hours = Var(within=NonNegativeReals, bounds=(0,8736), initialize=100)
off_fs.con_scale_zone_hours = Constraint(expr = off_fs.scaled_zone_hours*m.zone_total_hours == off_fs.zone_hours*8736)

#operating cost in $MM (million dollars)
m.op_expr = sum(op_zones[i].scaled_zone_hours*op_zones[i].fs.operating_cost for i in range(len(op_zones)))*1e-6 + \
off_fs.scaled_zone_hours*off_fs.fs.operating_cost*1e-6

#startup cost in MM$
m.startup_expr = m.startup_cst*m.nstartups*m.pmax*1e-6 #MM$

#set zone flowsheets to pyomo model
m.op_zones = op_zones

#Piecewise cost limits, connect marginal cost to operating cost. We say marginal cost is the average operating cost
m.connect_mrg_cost = Constraint(expr = m.marg_cst == 0.5*(op_zones[0].fs.operating_cost/m.pmin + op_zones[-1].fs.operating_cost/m.pmax))

# Expression for total cap and op cost - $
m.total_cost = Expression(expr=plant_lifetime*(m.op_expr  + m.startup_expr)+ capital_payment_years*cap_expr)

# Expression for total revenue
m.total_revenue = Expression(expr=plant_lifetime*m.revenue)

# Objective $
m.obj = Objective(expr=-(m.total_revenue - m.total_cost))

# Unfixing the boiler inlet flowrate for capex plant
m.cap_fs.fs.boiler.inlet.flow_mol[0].unfix()

# Setting bounds for the capex plant flowrate
m.cap_fs.fs.boiler.inlet.flow_mol[0].setlb(0.01)

# Setting bounds for net cycle power output for the capex plant
p_lower_bound=10
p_upper_bound=500
m.cap_fs.fs.eq_min_power = Constraint(
    expr=m.cap_fs.fs.net_cycle_power_output >= p_lower_bound*1e6)

m.cap_fs.fs.eq_max_power = Constraint(
    expr=m.cap_fs.fs.net_cycle_power_output <= p_upper_bound*1e6)

Creating instance  0
2022-03-09 16:23:44 [INFO] idaes.init.fs.boiler.control_volume: Initialization Complete
2022-03-09 16:23:44 [INFO] idaes.init.fs.boiler: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:44 [INFO] idaes.init.fs.turbine: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:44 [INFO] idaes.init.fs.pre_condenser.control_volume: Initialization Complete
2022-03-09 16:23:44 [INFO] idaes.init.fs.pre_condenser: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:44 [INFO] idaes.init.fs.condenser.control_volume: Initialization Complete
2022-03-09 16:23:44 [INFO] idaes.init.fs.condenser: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:44 [INFO] idaes.init.fs.bfw_pump.control_volume: Initialization Complete
2022-03-09 16:23:44 [INFO] idaes.init.fs.bfw_pump: Initialization Complete: optimal - Optimal Solution Found
2022-03-09 16:23:44 [INFO] idaes.init.fs.feed_water_heater.control_

## Setup Final Surrogate Inputs and Solve

We lastly fix a few surrogate inputs and solve the design problem. The solution of the design problem will determine market inputs that maximize the net plant revenue over 20 years.

In [26]:
# these are representative startup costs based on startup profiles we trained on.
# it is useful to fix the startup cost to one of these values since they are technically categorical variables
startup_csts = [0., 49.66991167, 61.09068702, 101.4374234,  135.2230393]

#fix some surrogate inputs
start_cst_index=2
m.startup_cst.fix(startup_csts[start_cst_index])
m.no_load_cst.fix(1.0)
m.min_up_time.fix(4.0)
m.min_dn_multi.fix(1.0)

solver = get_solver()
solver.options = {
    "tol": 1e-6
    #"mu_strategy": "adaptive"
}
status = solver.solve(m, tee=True)
sol_time = status['Solver'][0]['Time']

Ipopt 3.13.2: tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. S

  75r 2.5478658e+02 6.19e+04 8.49e+02   0.6 3.96e+06    -  1.00e+00 9.55e-02f  1
  76r 2.5084599e+02 1.37e+05 5.76e+02   0.6 3.55e+06    -  2.34e-01 1.15e-01f  1
  77r 2.5015843e+02 1.36e+05 5.63e+02   0.6 3.12e+06    -  6.53e-02 2.20e-02f  1
  78r 2.5014133e+02 1.36e+05 5.63e+02   0.6 3.41e+06    -  9.12e-02 7.84e-04f  1
  79r 2.4964161e+02 1.33e+05 8.90e+02   0.6 3.55e+06    -  1.00e+00 2.70e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r 2.4522299e+02 1.01e+05 1.49e+03   0.6 3.47e+06    -  1.13e-01 2.44e-01f  1
  81r 2.4149986e+02 7.38e+04 1.19e+03   0.6 2.63e+06    -  1.00e+00 2.71e-01f  1
  82r 2.4071119e+02 6.78e+04 9.85e+02   0.6 1.92e+06    -  1.00e+00 8.19e-02f  1
  83r 2.3343424e+02 1.16e+04 1.76e+02   0.6 1.77e+06    -  1.00e+00 8.28e-01f  1
  84r 2.3310517e+02 4.17e+03 2.23e+00  -0.1 3.04e+05    -  1.00e+00 1.00e+00f  1
  85r 2.3486409e+02 4.09e+03 3.51e+01  -1.5 5.99e+06    -  4.43e-01 4.94e-01f  1
  86r 2.3488169e+02 4.09e+03

 185r-3.0839215e+02 5.42e+05 9.99e+02   2.9 0.00e+00    -  0.00e+00 5.13e-09R  2
 186r-3.0942737e+02 8.91e+04 9.99e+02   2.9 8.78e+05    -  2.77e-04 9.75e-04f  1
 187 -3.0917146e+02 8.91e+04 2.26e+04  -1.0 1.04e+10    -  3.17e-07 5.58e-07f  5
 188 -3.0916599e+02 8.91e+04 2.26e+04  -1.0 6.96e+09    -  3.44e-07 1.24e-08h  8
 189r-3.0916599e+02 8.91e+04 9.99e+02   2.9 0.00e+00    -  0.00e+00 3.66e-07R  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r-3.0829920e+02 7.76e+04 1.89e+03   2.9 2.75e+05    -  1.46e-01 4.50e-04f  1
 191r-3.2460842e+02 2.27e+03 9.44e+03   2.9 2.75e+05    -  8.80e-01 1.48e-01f  1
 192r-3.6481711e+02 1.93e+03 2.26e+03   2.9 2.34e+05    -  9.90e-01 1.00e+00f  1
 193r-3.6093365e+02 1.59e+04 2.71e+02   2.9 2.01e+03    -  9.91e-01 1.00e+00f  1
 194r-3.6384190e+02 2.33e+03 8.16e+02   1.5 1.40e+00   2.0 8.18e-01 8.53e-01f  1
 195r-3.6395145e+02 1.58e+03 6.54e+02   0.8 1.25e+00   2.4 5.53e-01 3.24e-01f  1
 196r-3.6394153e+02 7.23e+02

 284r-2.4168477e+02 5.32e+03 9.99e+02   2.4 0.00e+00    -  0.00e+00 3.99e-07R  3
 285r-2.4210060e+02 8.24e+02 2.53e+04   2.4 2.86e+05    -  3.00e-01 2.13e-02f  1
 286r-2.5360530e+02 2.73e+02 2.80e+04   2.4 2.43e+05    -  9.90e-01 3.68e-01f  1
 287r-2.7190620e+02 2.34e+03 4.05e+03   2.4 1.50e+05    -  9.93e-01 1.00e+00f  1
 288r-2.7190620e+02 2.34e+03 9.99e+02   2.4 0.00e+00    -  0.00e+00 2.64e-07R  9
 289r-2.7191950e+02 2.21e+03 3.15e+04   2.4 2.71e+05    -  9.90e-01 3.84e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290r-2.7560262e+02 5.01e+03 5.33e+04   2.4 2.71e+05    -  9.92e-01 8.91e-01f  1
 291r-2.7472598e+02 2.36e+02 3.48e+02   2.4 2.92e+04    -  1.00e+00 1.00e+00f  1
 292r-2.7784691e+02 1.17e+03 5.45e+02   0.3 2.00e+04    -  8.61e-01 7.21e-01f  1
 293r-2.7958436e+02 2.95e+03 2.04e+04   0.3 2.74e+06    -  8.20e-01 4.05e-01f  1
 294r-2.7975005e+02 4.83e+03 3.50e+03   0.3 1.63e+06    -  1.00e+00 8.27e-01f  1
 295 -2.8561062e+02 4.83e+03

 380r-2.8104016e+02 5.96e+01 1.04e+04  -3.9 2.63e-02   2.7 8.27e-01 2.71e-01h  1
 381r-2.8104029e+02 5.96e+01 3.90e+03  -3.9 2.17e-03   3.1 3.08e-01 6.28e-01h  1
 382r-2.8104029e+02 5.96e+01 3.90e+03  -3.9 1.11e-02   2.7 7.65e-02 2.41e-04h  1
 383r-2.8103955e+02 5.96e+01 8.97e+03  -3.9 1.02e-01   2.2 2.34e-02 5.12e-01h  1
 384r-2.8103956e+02 5.96e+01 8.72e+03  -3.9 2.66e-02   2.6 8.38e-01 3.53e-02f  1
 385r-2.8103977e+02 5.96e+01 9.72e+03  -3.9 1.26e-02   2.1 5.73e-01 7.70e-02h  1
 386r-2.8104076e+02 5.96e+01 2.83e+03  -3.9 1.29e-02   2.6 2.95e-01 1.00e+00h  1
 387r-2.8104080e+02 5.96e+01 2.93e+03  -3.9 6.50e-02   2.1 2.51e-01 4.69e-03h  1
 388r-2.8104222e+02 5.96e+01 3.59e+00  -3.9 1.05e-02   2.5 1.00e+00 1.00e+00f  1
 389r-2.8104431e+02 5.96e+01 6.15e+02  -3.9 1.07e-01   2.0 4.04e-01 1.82e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390r-2.8104568e+02 5.96e+01 9.15e+02  -3.9 1.02e-02   2.5 9.06e-01 4.91e-01f  1
 391r-2.8104677e+02 5.96e+01

 519r-1.6337556e+02 2.03e+05 9.97e+02  -5.8 3.25e+07    -  3.47e-04 1.04e-07f 16
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 520r-1.6337563e+02 2.03e+05 9.97e+02  -5.8 3.25e+07    -  1.27e-04 1.04e-07f 16
 521r-1.6555582e+02 5.10e+04 9.97e+02  -5.8 3.25e+07    -  7.76e-05 3.41e-03f  1
 522r-1.6699826e+02 2.93e+04 9.96e+02  -5.8 3.24e+07    -  4.92e-05 3.49e-03f  1
 523r-1.6786999e+02 1.52e+04 9.96e+02  -5.8 3.23e+07    -  4.88e-04 4.08e-03f  1
 524r-1.6844940e+02 1.10e+04 9.37e+02  -5.8 3.21e+07    -  8.12e-02 7.85e-03f  1
 525r-1.5641904e+02 2.75e+04 9.37e+02  -5.8 3.19e+07    -  7.87e-06 6.26e-02f  1
 526r-1.4997025e+02 3.30e+04 8.64e+02  -5.8 2.99e+07    -  7.79e-02 4.57e-02f  1
 527r-1.4997024e+02 3.30e+04 6.36e+02  -5.8 2.85e+07    -  2.64e-01 8.23e-08f  1
 528r-1.4997020e+02 3.30e+04 9.54e+02  -5.8 2.85e+07    -  1.00e+00 3.65e-07f  1
 529r-1.3652964e+02 5.59e+04 8.35e+02  -5.8 2.85e+07    -  6.94e-01 1.25e-01f  1
iter    objective    inf_pr 

 642r-2.6599170e+01 1.30e+06 4.39e+04  -4.4 3.75e+06    -  1.00e+00 1.13e-01f  1
 643r-2.6666440e+01 1.28e+06 3.43e+04  -4.4 3.85e+06    -  2.14e-01 1.70e-02f  1
 644r-2.9908450e+01 2.31e+05 3.47e+04  -4.4 3.75e+06    -  9.84e-06 8.20e-01f  1
 645r-2.9908094e+01 2.28e+05 3.44e+04  -4.4 6.37e+01  -2.9 9.67e-03 1.41e-02h  1
 646r-2.8880250e+01 2.10e+05 3.11e+04  -4.4 6.12e+02  -3.3 9.91e-02 7.81e-02h  1
 647r-2.8867838e+01 2.10e+05 3.11e+04  -4.4 1.59e+06    -  1.42e-02 1.55e-03h  1
 648r-2.8559172e+01 2.02e+05 3.31e+04  -4.4 1.59e+06    -  1.00e+00 3.87e-02f  1
 649r-2.0236724e+01 7.35e+02 2.41e+02  -4.4 1.53e+06    -  5.29e-02 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650 -1.7989961e+01 4.38e+02 7.20e+04  -1.0 2.62e+08    -  5.13e-06 4.05e-03f  7
 651 -1.7724049e+01 4.35e+02 7.21e+04  -1.0 2.49e+08    -  3.51e-03 4.83e-04h 10
 652 -1.7473384e+01 4.32e+02 7.20e+04  -1.0 2.30e+08    -  1.62e-03 4.76e-04h 10
 653 -1.7242211e+01 4.30e+02

## Display Solution

In [28]:
#get value of surrogate inputs
x = [value(m.pmax),
    value(m.pmin_multi),
    value(m.ramp_multi),
    value(m.min_up_time),
    value(m.min_dn_multi),
    value(m.marg_cst),
    value(m.no_load_cst),
    value(m.startup_cst)
    ]

print("Market Inputs:", x)

#calculate revenues, costs, etc...
optimal_objective = -value(m.obj)

zone_hours = [value(m.zone_off.zone_hours)]
scaled_zone_hours = [value(m.zone_off.scaled_zone_hours)]
op_cost = [value(m.zone_off.fs.operating_cost)]
op_expr = value(m.zone_off.fs.operating_cost) # in dollars [$]
for zone in m.op_zones:
    zone_hours.append(value(zone.zone_hours))
    scaled_zone_hours.append(value(zone.scaled_zone_hours))
    op_cost.append(value(zone.fs.operating_cost))
    op_expr += value(zone.scaled_zone_hours)*value(zone.fs.operating_cost)

#more calculations of revenue and cost
revenue_per_year = value(m.revenue)
cap_expr = value(m.cap_fs.fs.capital_cost)/capital_payment_years
startup_expr = value(m.startup_expr)
total_cost = plant_lifetime*op_expr/1e6 + capital_payment_years*cap_expr
total_revenue = plant_lifetime*revenue_per_year

print("Annual Revenue [MM$]: ", revenue_per_year)
print("Capital Cost [MM$]: ",cap_expr)
print("Annual Startup Cost [MM$]: ", startup_expr)
print("Total Cost (20 years): ", total_cost)
print("Net Revenue (20 years)", optimal_objective)

Market Inputs: [177.5, 0.4262182413708828, 0.7858588589259599, 4.0, 1.0, 17.05604796538887, 1.0, 61.09068702]
Annual Revenue [MM$]:  30.095251533936473
Capital Cost [MM$]:  81.71386562461414
Annual Startup Cost [MM$]:  1.1412147244252577e-08
Total Cost (20 years):  834.075066129569
Net Revenue (20 years) -233.74143641981095
